In [1]:
import numpy
import pandas
import sklearn.ensemble
import sklearn.linear_model
import sklearn.model_selection

#import xgboost - Not working on Rob's laptop

print("Done")

Done


In [30]:
# General setup.

# Return mean absolute error scale.
def mean_absolute_error_salary_scale(y_test, y_predicted):
    return sklearn.metrics.mean_absolute_error(
        numpy.exp(y_test), numpy.exp(y_predicted))

In [7]:
# Define training and test data.

data = pandas.read_csv('data/train_large.csv')

X_train_index, X_test_index, Y_train, Y_test = sklearn.model_selection.train_test_split(
    data.index, data['LogSalaryNormalized'], test_size=.3, random_state=42)

# Define LDA data.
lda_data = pandas.read_csv('data/train_large_lda_50.csv')

X_train_lda = lda_data.iloc[X_train_index]
X_test_lda = lda_data.iloc[X_test_index]



In [13]:
# Model: Random Forests
# Data Representation: LDA
# Library used: sklearn

# NOTE: Have since retrained LDA using sklearn and online mini-batch learning.

rf = sklearn.ensemble.RandomForestRegressor(
    n_estimators=1000, n_jobs=-1, warm_start=True)

rf.fit(X_train_lda, Y_train)
rf_predictions = rf.predict(X_test_lda)

# Scale to get Mean Absolute Error
rf_mae = mean_absolute_error_salary_scale(Y_test, rf_predictions)

print('Random Forest Regressor Mean Absolute Error: {:10.4f}'.format(rf_mae))

Random Forest Regressor Mean Absolute Error:  8380.5428


### Interesting note: Using LDA, n_estimators = 10 performs better than 1000

In [ ]:
# Model: XGBoost
# Data Representation: LDA
# Library used: xgboost

# NOTE: xgboost will not import on Rob's laptop.

xgb = xgboost.XGBRegressor(
        learning_rate=.10, max_depth=10, n_estimators=1000, silent=False,
        subsample=.8, colsample_bytree=.8, nthread=4, min_child_weight=.5)

In [26]:
# Model: SGD
# Data Representation: LDA
# Library used: sklearn

# Normalization not required as LDA output is normalised between 0 and
# 1 anyway.

# We want a stochastic gradient descent with l1 norm.
sgd = sklearn.linear_model.SGDRegressor(
    alpha=.0001, penalty='l1', n_iter=10000)
sgd.fit(X_train_lda, Y_train)
sgd_predictions = sgd.predict(X_test_lda)
sgd_mae = mean_absolute_error_salary_scale(Y_test, sgd_predictions)
print('SGDRegressor Mean Absolute Error: {:10.4f}'.format(sgd_mae))
#TODO (any): wonder why this is so inaccurate/wrong

SGDRegressor Mean Absolute Error:  8929.7799
